In [1]:
import mnist_reader
from scipy.spatial import distance
import numpy as np
from sklearn.metrics.pairwise import euclidean_distances

In [117]:
eps = 5
minPoints = 5

In [3]:
training_images, training_labels = mnist_reader.load_mnist('data/fashion', kind='train')
test_images, test_labels = mnist_reader.load_mnist('data/fashion', kind='t10k')
images = np.concatenate((training_images, test_images), axis = 0)
labels = np.concatenate((training_labels, test_labels), axis = 0)

In [4]:
numbers = [0 for x in range(10)]
new_images = np.empty((0, 784), int)
new_labels = np.empty((0,), int)
i = 0
while(i < len(images)):
    numbers[labels[i]] += 1
    if numbers[labels[i]] <= 2000:
        new_images = np.append(new_images, np.array([images[i]]), axis=0)
        new_labels = np.append(new_labels, np.array([labels[i]]), axis=0)
    i += 1

images = new_images
labels = new_labels

In [5]:
distances = euclidean_distances(images)

In [83]:
neighbors_dict = {}
def create_neighbors():
    for i in range(len(images)):
        neighbors_dict[i] = []
        for j in range(len(images)):
            if((distances[i][j] / 256) <= eps):
                neighbors_dict[i].append(j)

In [84]:
create_neighbors()

In [91]:
total_list = []
for value in neighbors_dict.values():
    total_list.extend(value)
print("Mean: " + str(np.mean(total_list)))
print("Median: " + str(np.median(total_list)))
print("Min: " + str(np.min(total_list)))
print("Max: " + str(np.max(total_list)))

Mean: 8910.839376323818
Median: 8963.0
Min: 0
Max: 19999


In [115]:
visited = []
clusters = {}
noise = []
def dbscan():
    cluster = 0
    for index in range(len(images)):
        neighbors = list(set(neighbors_dict[index]))
        if len(neighbors) < minPoints: 
            noise.append(index)
        elif neighbors[0] not in visited:
            clusters[cluster] = []
            print(str(cluster), end = ": ")
            expandCluster(index, neighbors, cluster)
            print(len(clusters[cluster]))
            print("================================")
            cluster += 1

In [116]:
def expandCluster(index, neighbors, cluster):
    while(len(neighbors) != 0):
        point = neighbors.pop(0)
        point = int(point)
        if point not in visited:
            visited.append(point)
            clusters[cluster].append(point)
            potential_cluster_points = neighbors_dict[point]
            neighbors.extend(potential_cluster_points)
            neighbors = list(set(neighbors))
            

In [ ]:
def calculate_purity_of_clusters(clusters):
    total_purity = 0
    for i in range(0, len(clusters)):
        purity = purity_of_cluster(clusters[i])
        total_purity += purity
    total_purity = total_purity / len(images)
    return total_purity

In [ ]:
def purity_of_cluster(cluster):
    label_occurance = {}
    for image_index in cluster:
        label = labels[image_index]
        if label in label_occurance:
            label_occurance[label] += 1
        else:
            label_occurance[label] = 1
    majority_label = ""
    max_occurance = 0
    for label in label_occurance.keys():
        if label_occurance[label] > max_occurance:
            max_occurance = label_occurance[label]
            majority_label = label
    
    majority_count = 0
    for image_index in cluster:
        if labels[image_index] == majority_label:
            majority_count += 1
    
    return majority_count 

In [109]:
# minPoints = 50
dbscan()

0: 11653


In [103]:
# minPoints = 7
#eps = 5
dbscan()

0: 11653
1: 27


In [113]:
# minPoints = 20
#eps = 5
dbscan()

0: 11653


In [118]:
# minPoints = 5
#eps = 5
dbscan()

0: 11653
1: 9
2: 8
3: 6
4: 11
5: 8
6: 16
7: 14
8: 5
9: 27
10: 14
11: 11
12: 8
13: 9
14: 5
15: 10
16: 5
17: 8
18: 8


In [128]:
# minPoints = 5
#eps = 5
print(calculate_purity_of_clusters(clusters))

0.10785
